In [ ]:
!pip3 install langchain transformers openapi-pydantic pydantic==1.10.9 openapi-schema-pydantic
!pip3 install sentence-transformers opensearch-py

In [1]:
%load_ext autoreload
%autoreload 2

#### Index all functions from API

In [9]:
from data_processing import APISpecIndex
from sentence_transformers import SentenceTransformer

In [ ]:
specs_processor = APISpecIndex('qia3iowywamezen9p0mk.us-east-1.aoss.amazonaws.com', "api_functions")
embedding_model = SentenceTransformer("BAAI/bge-base-en-v1.5")
open_api_object = specs_processor.get_api_from_link("https://api.merge.dev/api/crm/v1/schema")


In [ ]:
specs_processor.index_documents(open_api_object, embedding_model)

In [12]:
specs_processor.opensearch_client.count()

{'count': 439,
 '_shards': {'total': 0, 'successful': 0, 'skipped': 0, 'failed': 0}}

#### Ask a user query

In [2]:
from conversation_manager import *

In [3]:
test_agent = ConversationAgent('qia3iowywamezen9p0mk.us-east-1.aoss.amazonaws.com', "api_functions", "BAAI/bge-base-en-v1.5")

RequestError(400, 'resource_already_exists_exception', 'OpenSearch exception [type=resource_already_exists_exception, reason=index [api_functions/9aNyoo4B75eYa7q9j_Dz] already exists]- server : [envoy]')


In [13]:
test_agent.run_query('Get total number of vacation days for employee id 134', None)

Retrieved total  10 results
function name /employees
function name /employee-payroll-runs/{id}
function name /employee-payroll-runs
function name /time-off
function name /employees/{id}
function name /time-off-balances
function name /timesheet-entries
function name /time-off-balances/{id}
function name /employments
function name /employments/{id}
API Functions selected by Language model:  To get the total number of vacation days for an employee with id 134, we need to follow these steps:

<result>
<reason>Get the time off balances for the employee</reason>
<function_name>/time-off-balances</function_name>
<input_parameters>
{
  "employee_id": "134",
  "policy_type": "VACATION"
}
</input_parameters>
</result>

<result>
<reason>From the list of time off balances, find the one with policy type 'VACATION' and get the 'total_balance' field value</reason>
<function_name>/time-off-balances/{id}</function_name>
<input_parameters>
{
  "id": "time_off_balance_id_with_policy_type_vacation",
  "ex

In [15]:
test_agent.run_query('How many employes do i have', None)

Retrieved total  10 results
function name /employees
function name /employments
function name /employments/{id}
function name /employee-payroll-runs
function name /employer-benefits
function name /employee-payroll-runs/{id}
function name /employees/{id}
function name /payroll-runs
function name /timesheet-entries
function name /employer-benefits/{id}
API Functions selected by Language model:  To answer the query "How many employees do I have?", we can use the `/employees` endpoint with the following steps:

<result>
<reason>Get a list of all employees</reason>
<function_name>/employees</function_name>
<input_parameters>
{
  "X-Account-Token": "Token identifying the end user"
}
</input_parameters>
</result>

This will return a list of `Employee` objects. We can then count the number of objects in the list to get the total number of employees.

No additional input parameters are required for this endpoint to retrieve the list of employees. The `X-Account-Token` header is needed to authen

### Assuming some of the above calls result in an answer

* {'function_name': '/time-off-balances', 'input_parameters': ' {   "employee_id": "134",   "policy_type": "VACATION" } '} ---> ```[{'function_name':'/time-off-balances','response':'34'}]```

* {'function_name': '/employees', 'input_parameters': ' {   "X-Account-Token": "Token identifying the end user" } '} ---> ```[{'function_name':'/employees','response':['employee 1', 'employee 2', 'employee 3']}]```

In [17]:
test_agent.run_query('How many employes do i have', None)

Retrieved total  10 results
function name /employees
function name /employments
function name /employments/{id}
function name /employee-payroll-runs
function name /employer-benefits
function name /employee-payroll-runs/{id}
function name /employees/{id}
function name /payroll-runs
function name /timesheet-entries
function name /employer-benefits/{id}


------ Final answer  ------


 {
  "reason": "The response from the '/employees' function contains a list of 3 employee names, which directly answers the question 'How many employees do I have?'",
  "answer": "You have 3 employees."
}


In [18]:
test_agent.run_query('Get total number of vacation days for employee id 134', None)

Retrieved total  10 results
function name /employees
function name /employee-payroll-runs/{id}
function name /employee-payroll-runs
function name /time-off
function name /employees/{id}
function name /time-off-balances
function name /timesheet-entries
function name /time-off-balances/{id}
function name /employments
function name /employments/{id}


------ Final answer  ------


 {
  "reason": "The response from the '/time-off-balances' function provides the total number of vacation days for the employee with ID 134, which directly answers the user's query.",
  "answer": "The total number of vacation days for employee ID 134 is 34."
}
